In [ ]:
import copy
import os
import numpy as np
from torchvision.datasets.folder import ImageFolder
from torchvision.datasets.folder import default_loader
import timm
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, SubsetRandomSampler
from sklearn.model_selection import StratifiedKFold
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
from sklearn.utils.class_weight import compute_class_weight
from torch.cuda.amp import autocast, GradScaler

In [ ]:
# Setting up data transformations and custom dataset class
def get_transforms():
    return transforms.Compose([

        transforms.Normalize(mean=[0.4965, 0.4965, 0.4964],
                             std=[0.2105, 0.2105, 0.2106])
    ])

In [ ]:
class CustomDataset(datasets.ImageFolder):
    def __getitem__(self, index):
        path, target = self.samples[index]
        sample = self.loader(path)
        sample = sample.convert("RGB")
        if self.transform is not None:
            sample = self.transform(sample)
        return sample, target

In [ ]:
# Dataset and DataLoader setup
def get_dataloaders(train_dir, val_dir, batch_size):
    train_transforms = get_transforms()
    val_transforms = get_transforms()

    train_dataset = CustomDataset(root=train_dir, transform=train_transforms)
    val_dataset = CustomDataset(root=val_dir, transform=val_transforms)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=16, pin_memory=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=16, pin_memory=True)

    return train_loader, val_loader, train_dataset

In [ ]:
def setup_model(model, device, num_classes):
    model = timm.create_model(model, pretrained=True, num_classes=num_classes)

    # No need to modify the final layer manually as in torchvision,
    # timm handles this with the num_classes parameter in create_model
    model = model.to(device)

    return model

In [ ]:
def calculate_class_weights(train_dataset):
    # Count the number of occurrences of each class
    class_counts = torch.zeros((len(train_dataset.classes)), dtype=torch.float)
    for _, label in train_dataset:
        class_counts[label] += 1

    # Compute class weights (inverse frequency or another method)
    class_weights = 1. / class_counts
    class_weights = (class_weights / class_weights.sum()) * len(train_dataset.classes)
    return class_weights

In [ ]:
# Validation function
def validate_model(model, val_loader, device):
    model.eval()  # Set the model to evaluation mode
    val_loss = 0.0
    val_corrects = 0
    criterion = nn.CrossEntropyLoss()

    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            with autocast():
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                _, preds = torch.max(outputs, 1)

            val_loss += loss.item() * inputs.size(0)
            val_corrects += torch.sum(preds == labels.data)

    epoch_loss = val_loss / len(val_loader.dataset)
    epoch_acc = val_corrects.double() / len(val_loader.dataset)
    return epoch_loss, epoch_acc

In [ ]:
# Training loop with mixed precision
def train_model(model_name, model, train_loader, device, epochs, class_weights):
    class_weights = torch.tensor(class_weights, dtype=torch.float).to(device)

    criterion = nn.CrossEntropyLoss(weight=class_weights)
    optimizer = optim.Adam(model.parameters(), lr=3e-5)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=15, gamma=0.01)
    scaler = GradScaler()

    best_val_loss = float('inf')
    best_model_state = None

    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()

            with autocast():
                outputs = model(inputs)
                loss = criterion(outputs, labels)

            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

            running_loss += loss.item() * inputs.size(0)

        scheduler.step()
        epoch_loss = running_loss / len(train_loader.dataset)
        val_loss, val_acc = validate_model(model, val_loader, device)

        print(f'Epoch {epoch+1}/{epochs}, Train Loss: {epoch_loss:.4f}, Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}')

        # Save the best model based on validation loss
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_model_state = model.state_dict()

    # Save the best model found during training
    if best_model_state:
        torch.save(best_model_state, f'{model_name}_final.pth')
        print("Saved the best model based on validation loss.")

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Using device: {torch.cuda.get_device_name(0)}')

val_dir = 'valid_data/'
train_dir = 'data/cleaned_images/cleaned_combined_images/'
batch_size = 16
num_classes = 3
epochs = 30

train_loader, val_loader, train_dataset = get_dataloaders(train_dir, val_dir, batch_size)

class_weights = calculate_class_weights(train_dataset)
class_weights = class_weights.to(device)

# model_resnet = setup_model('resnet50', device, num_classes)
model_efficientnet = setup_model('inception_resnet_v2', device, num_classes)
# train_model('resnet50', model_resnet, train_loader, device, epochs, class_weights)
train_model('inception_resnet_v2', model_efficientnet, train_loader, device, epochs, class_weights)

Using device: NVIDIA A100 80GB PCIe
Epoch 1/30, Train Loss: 0.2670, Val Loss: 0.7008, Val Acc: 0.6400
Epoch 2/30, Train Loss: 0.1278, Val Loss: 0.7970, Val Acc: 0.6700
Epoch 3/30, Train Loss: 0.0957, Val Loss: 0.6400, Val Acc: 0.7050
Epoch 4/30, Train Loss: 0.0765, Val Loss: 0.9688, Val Acc: 0.6700
Epoch 5/30, Train Loss: 0.0570, Val Loss: 0.8228, Val Acc: 0.7000
Epoch 6/30, Train Loss: 0.0363, Val Loss: 0.9099, Val Acc: 0.6900
Epoch 7/30, Train Loss: 0.0299, Val Loss: 0.7746, Val Acc: 0.7250
Epoch 8/30, Train Loss: 0.0268, Val Loss: 1.1664, Val Acc: 0.7100
Epoch 9/30, Train Loss: 0.0210, Val Loss: 0.9911, Val Acc: 0.7700
Epoch 10/30, Train Loss: 0.0185, Val Loss: 1.1532, Val Acc: 0.7300
Epoch 11/30, Train Loss: 0.0161, Val Loss: 1.0944, Val Acc: 0.6950
Epoch 12/30, Train Loss: 0.0175, Val Loss: 1.2109, Val Acc: 0.6850
Epoch 13/30, Train Loss: 0.0211, Val Loss: 1.0381, Val Acc: 0.7000
Epoch 14/30, Train Loss: 0.0144, Val Loss: 0.9846, Val Acc: 0.7200
Epoch 15/30, Train Loss: 0.0149, Va

/tmp/ipykernel_12652/2429079452.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  class_weights = torch.tensor(class_weights, dtype=torch.float).to(device)


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Using device: {torch.cuda.get_device_name(0)}')

val_dir = 'valid_data/'
train_dir = 'data/cleaned_images/cleaned_combined_images/'
batch_size = 16
num_classes = 3
epochs = 30

train_loader, val_loader, train_dataset = get_dataloaders(train_dir, val_dir, batch_size)

class_weights = calculate_class_weights(train_dataset)
class_weights = class_weights.to(device)

# model_resnet = setup_model('resnet50', device, num_classes)
model_efficientnet = setup_model('nasnetalarge', device, num_classes)
# train_model('resnet50', model_resnet, train_loader, device, epochs, class_weights)
train_model('nasnetalarge', model_efficientnet, train_loader, device, epochs, class_weights)

Using device: NVIDIA A100 80GB PCIe


model.safetensors:   0%|          | 0.00/356M [00:00<?, ?B/s]

/tmp/ipykernel_12652/2429079452.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  class_weights = torch.tensor(class_weights, dtype=torch.float).to(device)


Epoch 1/30, Train Loss: 0.2406, Val Loss: 0.6772, Val Acc: 0.6750
Epoch 2/30, Train Loss: 0.1112, Val Loss: 0.6313, Val Acc: 0.7100
Epoch 3/30, Train Loss: 0.0796, Val Loss: 0.6921, Val Acc: 0.7350
Epoch 4/30, Train Loss: 0.0467, Val Loss: 0.7840, Val Acc: 0.7400
Epoch 5/30, Train Loss: 0.0435, Val Loss: 0.7637, Val Acc: 0.6950
Epoch 6/30, Train Loss: 0.0245, Val Loss: 0.8776, Val Acc: 0.7150
Epoch 7/30, Train Loss: 0.0236, Val Loss: 0.8768, Val Acc: 0.7050
Epoch 8/30, Train Loss: 0.0143, Val Loss: 0.9930, Val Acc: 0.7300
Epoch 9/30, Train Loss: 0.0187, Val Loss: 1.0590, Val Acc: 0.7250
Epoch 10/30, Train Loss: 0.0100, Val Loss: 1.4519, Val Acc: 0.7350
Epoch 11/30, Train Loss: 0.0115, Val Loss: 1.4803, Val Acc: 0.6950
Epoch 12/30, Train Loss: 0.0168, Val Loss: 1.0150, Val Acc: 0.7300
Epoch 13/30, Train Loss: 0.0118, Val Loss: 1.3243, Val Acc: 0.7400
Epoch 14/30, Train Loss: 0.0109, Val Loss: 1.4470, Val Acc: 0.6850
Epoch 15/30, Train Loss: 0.0102, Val Loss: 1.6907, Val Acc: 0.7200
Epoc

In [ ]:
import torch
import timm

def load_model(model_name, checkpoint_path, num_classes):
    # Load the model structure from timm
    model = timm.create_model(model_name, pretrained=False, num_classes=num_classes)
    # Load the trained state dictionary
    model.load_state_dict(torch.load(checkpoint_path))
    model.eval()  # Set the model to evaluation mode
    return model

# Specify the path to your model checkpoints and number of classes
nasnet_checkpoint = 'nasnetalarge_final.pth'
inception_resnet_v2_checkpoint = 'inception_resnet_v2_final.pth'
num_classes = 3

# Load the models
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
nasnet_model = load_model('nasnetalarge', nasnet_checkpoint, num_classes).to(device)
inception_resnet_v2_model = load_model('inception_resnet_v2', inception_resnet_v2_checkpoint, num_classes).to(device)


In [ ]:
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

# Define the transformation, adjust according to how your model was trained
test_transform = transforms.Compose([
    transforms.Resize((600, 600)),
    transforms.CenterCrop(500),
    transforms.RandomRotation(degrees=10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.4965, 0.4965, 0.4964],
                             std=[0.2105, 0.2105, 0.2106]),
])

# Load the dataset
test_dir = 'Test'
test_dataset = ImageFolder(root=test_dir, transform=test_transform)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

In [ ]:
def predict_images_with_filenames(model, device, dataloader):
    model.eval()
    predictions = []
    filenames = []
    with torch.no_grad():
        for images, _ in dataloader:
            images = images.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            predictions.extend(predicted.cpu().numpy())
            # Capture filenames from the dataset
            filenames.extend([dataloader.dataset.samples[i][0] for i in range(len(images))])

    return filenames, predictions

filenames_nasnet, predictions_nasnet = predict_images_with_filenames(nasnet_model, device, test_loader)
filenames_inception, predictions_inception = predict_images_with_filenames(inception_resnet_v2_model, device, test_loader)


In [ ]:
import pandas as pd

base_filenames_nasnet = [os.path.basename(f) for f in filenames_nasnet]
base_filenames_inception = [os.path.basename(f) for f in filenames_inception]

# Create DataFrame
results_df_inception = pd.DataFrame({
    'Filename': base_filenames_inception,
    'Prediction': predictions_inception
})

results_df_nasnet = pd.DataFrame({
    'Filename': base_filenames_nasnet,
    'Prediction': predictions_nasnet
})

# Save to Excel
path_inception = 'predicted_results_inception.csv'
path_nasnet = 'predicted_results_nasnet.csv'
results_df_inception.to_csv(path_inception, index=False, header=False)
results_df_nasnet.to_csv(path_nasnet, index=False, header=False)

print(f"NasNetLarge results saved to {path_inception}")
print(f"InceptionResNetV2 results saved to {path_inception}")

NasNetLarge results saved to predicted_results_inception.csv
InceptionResNetV2 results saved to predicted_results_inception.csv


In [ ]:
from azureml.core import Workspace
ws = Workspace.from_config()  # Or use .get() with explicit parameters
compute_target = ws.compute_targets['akrishn21']
compute_target.stop(show_output=True)